<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [1]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler, scale

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap

#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-psmqht26/mightylogic_a02b1f5cb0864309805218ae113b45aa
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-psmqht26/mightylogic_a02b1f5cb0864309805218ae113b45aa
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=8693 sha256=4ee57aadab47e1d48361a56b053731fbe9621ac642dbdf1f1973bb24f7c3fbe6
  Stored in directory: /tmp/pip-ephem-wheel-cache-rq951fzf/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic


# Turf War Demo

In [2]:

#
# A Turf War Map
#

map2 =TurfWarMap()
map2.setGuild("Nightterror")
map2.setStartDate(year=2021, month=7, day=17)


# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [3]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [4]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [5]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,112585.5,135011.0,154539.6,135011.0,120009.6,166503.0
B,0.0,297125.5,0.0,274546.6,381068.0,0.0
C,187590.5,294048.0,334550.0,429130.0,0.0,357080.0
D,174041.0,0.0,358603.0,0.0,390143.0,0.0
E,60006.0,174041.0,135011.0,199566.0,135011.0,111023.0


In [6]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

fig1.show()
fig2.show()

In [7]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution



In [8]:
#
# List of buildings easy for players to read.
#
map2.printBuildingList()

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


In [9]:
#
# Text report that highlights building returns 
#

map2.printReport()

Turf War Map Report
Guild: Nightterror
For the Week of: 17 July 2021
Total Map Value: 1,837,876 common souls

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


In [10]:
#
# Evaluate neighbors of a given tile
#
map2.printNeighborList('B',2)

Neighbors of tile B:2
Building          Points      For 1st Place                      Guild Gets
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
A-2: Forest        30,003   210 common souls, 140 contribution,    0 influence.
C-2: Plains        75,005   350 common souls, 350 contribution,    0 influence.


# Discrete Solution to Find Next Move

Played around with an analytical solution, but it was too problemmatic.

In [11]:
#
# High values are desirable payout locations, low are less desirable
#

pd.DataFrame(map2.getScaledPayout())

,0,1,2,3,4,5
0,0.404740,0.404740,0.404740,0.000000,0.404740,0.404740
1,1.823342,1.618900,2.550525,1.618900,1.618900,3.278044
2,0.404740,1.618900,1.618900,1.618900,3.360603,0.404740
3,0.404740,2.671935,1.618900,3.360603,1.618900,1.781186
4,0.404740,0.404740,0.404740,0.404740,0.404740,0.404740


In [12]:
#
# High values are desirable staging locations, low are less desirable
#
pd.DataFrame(map2.getScaledStaging())

,0,1,2,3,4,5
0,0.282264,0.338487,0.387447,0.338487,0.300877,0.417441
1,0.000000,0.744925,0.000000,0.688317,0.955378,0.000000
2,0.470309,0.737209,0.838752,1.075874,0.000000,0.895237
3,0.436339,0.000000,0.899056,0.000000,0.978130,0.000000
4,0.150441,0.436339,0.338487,0.500333,0.338487,0.278346


In [13]:
#
# Lowest values are least desirable staging locations, higher are better staging locations, highest are desirable payout locations
#
adjusted_move = map2.getAdjustedMoves()
#adjusted_move = np.maximum(map2.getScaledPayout(),map2.getScaledStaging())
#adjusted_move = map2.getScaledPayout() + map2.getScaledStaging()
#adjusted_move = map2.getScaledPayout() - map2.getScaledStaging()

pd.DataFrame(adjusted_move)

,0,1,2,3,4,5
0,0.066253,0.066253,0.066253,0.000000,0.066253,0.078954
1,1.484856,1.280413,2.212038,1.280413,1.280413,2.939557
2,0.131822,1.280413,1.280413,1.280413,3.022117,0.556750
3,0.097852,2.333448,1.280413,3.022117,1.280413,1.442699
4,0.066253,0.097852,0.066253,0.161846,0.066253,0.066253


In [14]:
moves = adjusted_move
get_heatmap(moves,"Movement Array", round=False).show()

# Notes

1.   Why is E:1 higher than D:1 and E:2?  It has the same payout and a lower staging value.  Therefore, payout-staging is higher.
2.   D:6 looks funny, but checks out



In [15]:
#
# 3D Presentation of the movement surface
#
flip = np.flip( moves, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])
fig.update_layout(title='Movement Array in 3D', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [16]:

flip = moves
epsilon = 0.05
rows = range(5)
cols = range(6)
retx = [[0 for i in cols] for j in rows]
rety = [[0 for i in cols] for j in rows]

for row in rows:
  for col in cols:
    up = row +1
    down = row -1
    left = col -1
    right = col +1

    me = flip[row][col]
    max = me
    z_up = -1000000
    z_dn = -1000000
    z_left = -1000000
    z_right = -1000000

    if up in rows:
      tmp = flip[up][col]
      z_up = tmp
      if tmp> max:
        max = tmp
    if down in rows:
      tmp = flip[down][col]
      z_dn = tmp
      if tmp> max:
        max = tmp        
    if left in cols:
      tmp = flip[row][left]
      z_left = tmp
      if tmp> max:
        max = tmp     
    if right in cols:
      tmp = flip[row][right]
      z_right= tmp
      if tmp> max:
        max = tmp     

    drow = 4-row
    if max == me:
      retx[drow][col] = 0
      rety[drow][col] = 0
    elif max == z_left:
      retx[drow][col] = -1
      rety[drow][col] = 0      
    elif max == z_right:
      retx[drow][col] = 1
      rety[drow][col] = 0    
    elif max == z_up:
      retx[drow][col] = 0
      rety[drow][col] = -1    
    elif max == z_dn:
      retx[drow][col] = 0
      rety[drow][col] = 1  

# pd.DataFrame(retx)
# pd.DataFrame(rety)

In [17]:
  #fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  x,y = np.meshgrid(np.arange(0, 6, 1), np.arange(0, 5, 1))
  fig = ff.create_quiver(x,y, retx, rety, scale=1, name="Recommended Move",
                      line=dict(width=2,
                                color='red'),
                      )
   #) #, x_start=0, x_end=6)


  cnames = []
  rtl = ['E','D','C','B','A']
  ctli = [1,2,3,4,5,6]
  for c in ctli:
    for r in rtl:
      cnames.append(f"{r}{c}")
      #print(f"{r}{c}")

  # add dots at integers
  fig.add_trace(go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers+text',
                          textposition="top right",
                          text = cnames,
                          textfont=dict(
                            family="sans serif",
                            size=18,
                            color="red"
                          ),
                          marker_size=14,
                          name='Tiles'))
  #x axis
  fig.update_xaxes(visible=False)

  #y axis    
  z = payout.to_numpy()
  z = np.flip( z, axis=0)
  fig.update_yaxes(visible=False)
  fig.add_trace(    go.Contour(
        z=z,
        x=[0,1,2,3,4,5], # horizontal axis
        y=[0, 1, 2,3,4], # vertical axis
        colorscale="delta"
        #showlegend=False
        #color_continuous_scale=px.colors.sequential.Viridis
    ))
  
  #fig.update_layout dict(show_legend=False))
  fig.update_layout(dict(paper_bgcolor='White',
                      plot_bgcolor='White', showlegend=False))
  fig.update_layout(coloraxis_showscale=False)

  #fig.add_shape(type="rect",
  #  xref="x", yref="y",
  #  x0=-0.5, y0=-0.5,
  #  x1=0.5, y1=0.5,
  #  line=dict(
  #      color="RoyalBlue",
  #      width=3,
  #  ),
  #  #fillcolor="LightSkyBlue",
  # #)
  # move colorbar
  fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                          ticks="outside",
                                          ticksuffix=" bills"))
  fig.update_layout(title='Relative Payout and Recommended Moves')
  fig.show()

In [18]:
import re
s = 'a1a5c6e3'
res = re.findall('(\d+|[A-Za-z]+)', s)
print(res)

def parse_tile(inp):
  pass

['a', '1', 'a', '5', 'c', '6', 'e', '3']


# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
1. Add payout report for multiple tiles

